In [ ]:
# pip install moviepy
# pip install opencv-python-headless pyaudio moviepy

In [19]:
import sounddevice as sd

sd.query_devices() 

   0 Microsoft Sound Mapper - Input, MME (2 in, 0 out)
>  1 Microphone (Realtek(R) Audio), MME (2 in, 0 out)
   2 Microsoft Sound Mapper - Output, MME (0 in, 2 out)
<  3 Speakers (Realtek(R) Audio), MME (0 in, 2 out)
   4 Primary Sound Capture Driver, Windows DirectSound (2 in, 0 out)
   5 Microphone (Realtek(R) Audio), Windows DirectSound (2 in, 0 out)
   6 Primary Sound Driver, Windows DirectSound (0 in, 2 out)
   7 Speakers (Realtek(R) Audio), Windows DirectSound (0 in, 2 out)
   8 Realtek ASIO, ASIO (2 in, 2 out)
   9 Speakers (Realtek(R) Audio), Windows WASAPI (0 in, 2 out)
  10 Microphone (Realtek(R) Audio), Windows WASAPI (2 in, 0 out)
  11 Microphone (Realtek HD Audio Mic input), Windows WDM-KS (2 in, 0 out)
  12 Speakers (Realtek HD Audio output), Windows WDM-KS (0 in, 2 out)
  13 Stereo Mix (Realtek HD Audio Stereo input), Windows WDM-KS (2 in, 0 out)

# Version 1

In [13]:
import cv2
import pyaudio
import wave
from moviepy.editor import VideoFileClip, AudioFileClip

In [41]:
# Define video capture paramters
cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 30, (640, 480))


# Define audio recording parameters
FORMAT = pyaudio.paInt16  # Audio format (16-bit PCM)
CHANNELS = 1  # Number of audio channels (1 for mono, 2 for stereo)
RATE = 30000  # Sample rate (samples per second) # 30000 for 30 fps on this laptop
CHUNK = 1024  # Number of frames per buffer

# Set the volume of the input device (change index to your input device)
input_device_index = 1  # Change this to your input device index
input_volume = 1  # Adjust this value (0.0 to 1.0) to set volume

audio = pyaudio.PyAudio()
stream = audio.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK,
                    input_device_index = input_device_index)

stream.input_device_gain = input_volume

frames = []
print("Recording...")

record_audio = True
while cap.isOpened() and record_audio:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Display the video frame in a window
    cv2.imshow('Video', frame)
    
    # Write video frames to the output file
    out.write(frame)

    # Record audio frames continuously until the video is being recorded
    data = stream.read(CHUNK)
    frames.append(data)
    
    # Check for the 'q' key press to stop recording
    if cv2.waitKey(1) & 0xFF == ord('q'):
        record_audio = False
        break

print("Finished recording.")

# Release video resources
out.release()
cap.release()
cv2.destroyAllWindows()

# Stop and close the audio stream
stream.stop_stream()
stream.close()
audio.terminate()

# Save the recorded audio to a WAV file
audio_file = "audio.wav"
with wave.open(audio_file, 'wb') as wf:
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))

# Combine video and audio using moviepy
video = VideoFileClip('output.avi').set_fps(30)
audio = AudioFileClip(audio_file)

video = video.set_audio(audio)
output_path = 'final_output.mp4'
video.write_videofile(output_path, codec='libx264', audio_codec='aac')

print("Finished saving final video.")

Recording...
Finished recording.
Moviepy - Building video final_output.mp4.
MoviePy - Writing audio in final_outputTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video final_output.mp4



Moviepy - Done !
Moviepy - video ready final_output.mp4
Finished saving final video.


# Version 2

In [16]:
import cv2
import pyaudio
import wave
import numpy as np

# Initialize video capture
video_capture = cv2.VideoCapture(0)  # You can change this to your video file

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_output = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))  # Adjust resolution as needed

# Audio settings
audio_format = pyaudio.paInt16
audio_channels = 1
audio_rate = 44100
audio_chunk = 1024

# Initialize audio recording
audio = pyaudio.PyAudio()
audio_stream = audio.open(format=audio_format,
                          channels=audio_channels,
                          rate=audio_rate,
                          input=True,
                          frames_per_buffer=audio_chunk)

audio_frames = []

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Display video frame
    cv2.imshow('Video', frame)
    
    # Record audio
    audio_data = audio_stream.read(audio_chunk)
    audio_frames.append(audio_data)

    # Write video frame to output file
    video_output.write(frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything when done
video_capture.release()
video_output.release()
cv2.destroyAllWindows()

# Save audio to a WAV file
audio_stream.stop_stream()
audio_stream.close()
audio.terminate()

with wave.open('audio_output.wav', 'wb') as wf:
    wf.setnchannels(audio_channels)
    wf.setsampwidth(audio.get_sample_size(audio_format))
    wf.setframerate(audio_rate)
    wf.writeframes(b''.join(audio_frames))
